# సెషన్ 2 – కనిష్ట RAG పైప్‌లైన్

Foundry Local + sentence-transformers embeddings ఉపయోగించి ఒక తేలికపాటి Retrieval-Augmented Generation పైప్‌లైన్‌ను నిర్మించండి.


### వివరణ: డిపెండెన్సీ ఇన్‌స్టాలేషన్
ఈ పైప్‌లైన్ కోసం కనీస ప్యాకేజీలను ఇన్‌స్టాల్ చేస్తుంది:
- స్థానిక మోడల్ నిర్వహణ కోసం `foundry-local-sdk` (ప్యూర్ BASE_URL మార్గం ఉపయోగించకపోతే).
- అనుకూల SDK నిర్మాణాల కోసం `openai` (కొన్ని యుటిలిటీల కోసం).
- ఎంబెడ్డింగ్స్ కోసం `sentence-transformers`.
- వెక్టర్ గణితానికి `numpy`.
పునఃప్రారంభించడం సురక్షితం; వాతావరణం ఇప్పటికే సంతృప్తి చెందితే దాటవేయండి.


# పరిస్థితి
ఈ నోట్‌బుక్ పూర్తిగా స్థానికంగా నడిచే కనిష్ట రిట్రీవల్-ఆగ్మెంటెడ్ జనరేషన్ (RAG) పైప్‌లైన్‌ను నిర్మిస్తుంది:
- ఫౌండ్రీ లోకల్ మోడల్‌కు కనెక్ట్ అవుతుంది (SDK లేదా BASE_URL ద్వారా ఆటో-డిటెక్ట్ చేస్తుంది).
- ఒక చిన్న ఇన్-మెమరీ డాక్యుమెంట్ కార్పస్ సృష్టించి దానిని సెంటెన్స్ ట్రాన్స్‌ఫార్మర్స్‌తో ఎంబెడ్ చేస్తుంది.
- పారదర్శకత కోసం సాదాసీదా వెక్టర్ సారూప్యత రిట్రీవల్‌ను అమలు చేస్తుంది (బాహ్య సూచిక లేదు).
- బహుళ HTTP ఫాల్‌బ్యాక్ మార్గాల ద్వారా గ్రౌండెడ్ జనరేషన్ అభ్యర్థనలను జారీ చేస్తుంది (`/v1/chat/completions`, `/v1/completions`, `/v1/responses`).
- ప్రారంభ ప్రయత్నాలు విఫలమైనప్పుడు ప్రత్యామ్నాయ మోడల్ రూపాలను మళ్లీ ప్రయత్నించే `answer()` సహాయకాన్ని అందిస్తుంది.

ఇది పెద్ద కార్పసులకు, స్థిరమైన వెక్టర్ స్టోర్లకు లేదా మూల్యాంకన ప్రమాణాలకు (RAG మూల్యాంకన నోట్‌బుక్ చూడండి) విస్తరించే ముందు డయాగ్నోస్టిక్ టెంప్లేట్‌గా ఉపయోగించండి.


In [5]:
# Install dependencies
!pip install -q foundry-local-sdk openai sentence-transformers numpy

### వివరణ: కోర్ ఇంపోర్ట్స్
ఎంబెడ్డింగ్ + లోకల్ ఇన్ఫరెన్స్ కోసం అవసరమైన కోర్ లైబ్రరీలను లోడ్ చేస్తుంది:
- సెంటెన్స్‌ట్రాన్స్‌ఫార్మర్ డెన్స్ వెక్టర్ ఎంబెడ్డింగ్స్ కోసం.
- ఫౌండ్రీలోకల్ మేనేజర్ (ఐచ్ఛికం) లోకల్ సర్వీస్‌ను నిర్వహించడానికి.
- ఓపెన్‌ఏఐ క్లయింట్ పరిచయమైన ఆబ్జెక్ట్ ఆకృతుల కోసం (తరువాత మేము నేరుగా HTTP ను ఉపయోగించినప్పటికీ).


In [6]:
import os, numpy as np
from sentence_transformers import SentenceTransformer
from foundry_local import FoundryLocalManager
from openai import OpenAI

### వివరణ: టాయ్ డాక్యుమెంట్ కార్పస్
డొమైన్ స్టేట్మెంట్ల చిన్న ఇన్-మెమరీ జాబితాను నిర్వచిస్తుంది. పునరావృతిని వేగంగా మరియు నియంత్రితంగా ఉంచుతుంది కాబట్టి దృష్టి డేటా నిర్వహణ కాకుండా పైప్‌లైన్ యాంత్రికతలపై (పునఃప్రాప్తి + గ్రౌండింగ్) ఉంటుంది.


In [7]:
DOCS = [
    'Foundry Local provides an OpenAI-compatible local inference endpoint.',
    'Retrieval Augmented Generation improves answer grounding by injecting relevant context.',
    'Edge AI reduces latency and preserves privacy via local execution.',
    'Small Language Models can offer competitive quality with lower resource usage.',
    'Vector similarity search retrieves semantically relevant documents.'
]

### వివరణ: కనెక్షన్, మోడల్ ఎంపిక & ఎంబెడ్డింగ్ ప్రారంభం
దృఢమైన కనెక్షన్ లాజిక్:
1. ఐచ్ఛికంగా స్పష్టమైన `BASE_URL` (ప్యూర్ HTTP పాథ్) ఉపయోగిస్తుంది లేకపోతే FoundryLocalManager కిFallback అవుతుంది.
2. `/v1/models` ను ప్రోబ్ చేసి ఉత్తమ సరిపోయే కాంక్రీట్ మోడల్ ID ను ఎంచుకుంటుంది (ఖచ్చితమైన అలియాస్ > కానానికల్ ఫ్యామిలీ > మొదటి అందుబాటులో ఉన్నది).
3. కాంఫిగర్ చేయదగిన `FOUNDRY_CONNECT_RETRIES` & డిలే తో రీట్రై లూప్ అమలు చేస్తుంది.
4. టాయ్ కార్పస్ కోసం SentenceTransformer ఎంబెడ్డింగ్స్ (నార్మలైజ్డ్ వెక్టార్లు) ను ప్రారంభిస్తుంది.
5. పునరుత్పత్తి కోసం OpenAI SDK వెర్షన్ ను క్యాప్చర్ చేస్తుంది.
సేవ అందుబాటులో లేకపోతే, క్రాష్ కాకుండా దాన్ని ప్రారంభించడానికి మార్గదర్శకాన్ని ప్రింట్ చేస్తుంది.


In [12]:
import os, time, json, requests, re
# Native Foundry Local SDK preferred; fall back to explicit BASE_URL if provided
os.environ.setdefault('FOUNDRY_LOCAL_ALIAS', 'phi-4-mini')
alias = os.getenv('FOUNDRY_LOCAL_ALIAS', os.getenv('TARGET_MODEL', 'phi-4-mini'))
base_url_env = os.getenv('BASE_URL', '').strip()
manager = None
client = None
endpoint = None

def _canonicalize(model_id: str) -> str:
    """Remove CUDA suffix and version tags from model name."""
    b = model_id.split(':')[0]
    return re.sub(r'-cuda.*', '', b)

try:
    if base_url_env:
        # Allow user override; normalize by removing trailing / and optional /v1
        root = base_url_env.rstrip('/')
        if root.endswith('/v1'):
            root = root[:-3]
        endpoint = root
        print(f'[INFO] Using explicit BASE_URL override: {endpoint}')
    else:
        from foundry_local import FoundryLocalManager
        manager = FoundryLocalManager(alias)
        # Manager endpoint already includes /v1 - remove it for our base
        raw_endpoint = manager.endpoint.rstrip('/')
        if raw_endpoint.endswith('/v1'):
            endpoint = raw_endpoint[:-3]
        else:
            endpoint = raw_endpoint
        print(f'[OK] Foundry Local manager endpoint: {manager.endpoint} | base={endpoint} | alias={alias}')
    
    # Probe models list (endpoint does NOT include /v1 here)
    models_resp = requests.get(endpoint + '/v1/models', timeout=5)
    models_resp.raise_for_status()
    payload = models_resp.json() if models_resp.headers.get('content-type','').startswith('application/json') else {}
    data = payload.get('data', []) if isinstance(payload, dict) else []
    ids = [m.get('id') for m in data if isinstance(m, dict)]
    
    # Select best matching model
    chosen = None
    if alias in ids:
        chosen = alias
    else:
        for mid in ids:
            if _canonicalize(mid) == _canonicalize(alias):
                chosen = mid
                break
    if not chosen and ids:
        chosen = ids[0]
    model_name = chosen or alias
    
    # Initialize OpenAI client
    from openai import OpenAI as _OpenAI
    client = _OpenAI(
        base_url=endpoint + '/v1',  # OpenAI client needs full base URL with /v1
        api_key=(getattr(manager, 'api_key', None) or os.getenv('API_KEY') or 'not-needed')
    )
    print(f'[OK] Model resolved: {model_name} (total_models={len(ids)})')
except Exception as e:
    print('[ERROR] Failed to initialize Foundry Local client:', e)
    client = None
    model_name = alias

# Expose BASE for downstream compatibility (without /v1)
BASE = endpoint

# Embeddings setup
embed_model_name = os.getenv('EMBED_MODEL', 'sentence-transformers/all-MiniLM-L6-v2')
try:
    from sentence_transformers import SentenceTransformer
    embedder = SentenceTransformer(embed_model_name)
    doc_emb = embedder.encode(DOCS, convert_to_numpy=True, normalize_embeddings=True)
    print(f'[OK] Embedded {len(DOCS)} docs using {embed_model_name} shape={doc_emb.shape}')
except Exception as e:
    print('[ERROR] Embedding init failed:', e)
    embedder = None
    doc_emb = None

try:
    import openai as _openai
    openai_version = getattr(_openai, '__version__', 'unknown')
    print('OpenAI SDK version:', openai_version)
except Exception:
    openai_version = 'unknown'

if client is None:
    print('\nNEXT: Start/verify service then re-run this cell:')
    print('  foundry service start')
    print('  foundry model run phi-4-mini')
    print('  (optional) set BASE_URL=http://127.0.0.1:57127')

[OK] Foundry Local manager endpoint: http://127.0.0.1:59778/v1 | base=http://127.0.0.1:59778 | alias=phi-4-mini
[OK] Model resolved: deepseek-r1-distill-qwen-7b-cuda-gpu:0 (total_models=11)
[OK] Embedded 5 docs using sentence-transformers/all-MiniLM-L6-v2 shape=(5, 384)
OpenAI SDK version: 1.109.1


### వివరణ: రిట్రీవ్ ఫంక్షన్ (వెక్టర్ సారూప్యత)
`retrieve(query, k=3)` క్వెరీని ఎన్‌కోడ్ చేస్తుంది, కోసైన్ సారూప్యతను (నార్మలైజ్డ్ వెక్టర్లపై డాట్ ప్రొడక్ట్) లెక్కిస్తుంది, మరియు టాప్-k డాక్ సూచికలను తిరిగి ఇస్తుంది. ఇది పారదర్శకత కోసం కనిష్టంగా మరియు మెమరీలోనే ఉంటుంది.


In [9]:
def retrieve(query, k=3):
    q = embedder.encode([query], convert_to_numpy=True, normalize_embeddings=True)[0]
    sims = doc_emb @ q
    return sims.argsort()[::-1][:k]

### వివరణ: SDK-ఆధారిత ఉత్పత్తి & జవాబు సహాయకుడు
మాన్యువల్ రా HTTP పోస్ట్‌ల స్థానంలో Foundry Local SDK + OpenAI-అనుకూల క్లయింట్ పద్ధతులను ఉపయోగించడానికి పునఃరూపకల్పన చేయబడింది:
- ప్రాథమిక మార్గం: `client.chat.completions.create` (సంరచిత సందేశాలు).
- ప్రత్యామ్నాయాలు: `client.completions.create` (పాత ప్రాంప్ట్) తరువాత `client.responses.create` (సులభతరమైన ప్రతిస్పందనలు API).
- ప్రత్యామ్నాయ మోడల్ IDలను (RAW vs తొలగించిన ALT) సాధారణీకరించి అనుకూలతను విస్తరించును.
- `answer()` టాప్-k పొందిన డాక్యుమెంట్ల నుండి ఒక స్థిరమైన ప్రాంప్ట్‌ను నిర్మించి, ఆర్డర్ చేసిన ప్రయత్న ట్రేస్‌లను నమోదు చేస్తుంది.
ఇది లాజిక్‌ను చదవడానికి సులభంగా ఉంచుతూ, అభివృద్ధి చెందుతున్న OpenAI-అనుకూల ఎండ్‌పాయింట్లలో సున్నితమైన తగ్గింపును అందిస్తుంది.


In [14]:
# SDK-based generation (Foundry Local manager + OpenAI client methods)
import re, time, json

def _strip_model_name(name: str) -> str:
    """Strip CUDA suffix and version tags from model name."""
    base = name.split(':')[0]
    base = re.sub(r'-cuda.*', '', base)
    return base

# Use the actual resolved model name from connection cell
RAW_MODEL = model_name
ALT_MODEL = _strip_model_name(RAW_MODEL)

def _try_via_client(messages, prompt, model_id: str, max_tokens=220, temperature=0.2):
    """Try generating response using OpenAI client with multiple fallback routes."""
    attempts = []
    
    # 1. Try chat.completions endpoint (preferred for chat models)
    try:
        resp = client.chat.completions.create(
            model=model_id, 
            messages=messages, 
            max_tokens=max_tokens, 
            temperature=temperature
        )
        content = resp.choices[0].message.content
        attempts.append(('chat.completions', 200, (content or '')[:160]))
        if content and content.strip():
            return content, attempts
    except Exception as e:
        attempts.append(('chat.completions', None, str(e)[:160]))
    
    # 2. Try legacy completions endpoint
    try:
        comp = client.completions.create(
            model=model_id, 
            prompt=prompt, 
            max_tokens=max_tokens, 
            temperature=temperature
        )
        txt = comp.choices[0].text if comp.choices else ''
        attempts.append(('completions', 200, (txt or '')[:160]))
        if txt and txt.strip():
            return txt, attempts
    except Exception as e:
        attempts.append(('completions', None, str(e)[:160]))
    
    return None, attempts

def retrieve(query, k=3):
    """Retrieve top-k most similar documents using cosine similarity."""
    if embedder is None or doc_emb is None:
        raise RuntimeError("Embeddings not initialized.")
    q_emb = embedder.encode([query], normalize_embeddings=True)[0]
    scores = doc_emb @ q_emb
    idxs = np.argsort(scores)[::-1][:k]
    return idxs

def answer(query, k=3, max_tokens=220, temperature=0.2, try_alternate=True):
    """
    Answer a query using RAG pipeline:
    1. Retrieve relevant documents using vector similarity
    2. Generate grounded response using Foundry Local model via OpenAI SDK
    
    Args:
        query: User question
        k: Number of documents to retrieve
        max_tokens: Maximum tokens for generation
        temperature: Sampling temperature
        try_alternate: Whether to try alternate model name on failure
    
    Returns:
        Dictionary with query, answer, docs, context, route, and tried attempts
    """
    if client is None:
        raise RuntimeError('Model client not initialized. Re-run connection cell after starting Foundry Local.')
    if embedder is None or doc_emb is None:
        raise RuntimeError('Embeddings not initialized.')
    
    # Retrieve relevant documents
    idxs = retrieve(query, k=k)
    context = '\n'.join(f'Doc {i}: {DOCS[i]}' for i in idxs)
    
    # Construct grounded generation prompt
    system_content = 'Use ONLY provided context. If insufficient, say "I\'m not sure."'
    user_content = f'Context:\n{context}\n\nQuestion: {query}'
    messages = [
        {'role': 'system', 'content': system_content},
        {'role': 'user', 'content': user_content}
    ]
    prompt = f'System: {system_content}\n{user_content}\nAnswer:'
    
    # Try generation with primary model
    tried = []
    ans, attempts = _try_via_client(messages, prompt, RAW_MODEL, max_tokens=max_tokens, temperature=temperature)
    tried.append({'model': RAW_MODEL, 'attempts': attempts})
    
    if ans and ans.strip():
        return {
            'query': query, 
            'answer': ans.strip(), 
            'docs': idxs.tolist(), 
            'context': context, 
            'route': 'chat-first', 
            'tried': tried
        }
    
    # Try alternate model name if available
    if try_alternate and ALT_MODEL != RAW_MODEL:
        ans2, attempts2 = _try_via_client(messages, prompt, ALT_MODEL, max_tokens=max_tokens, temperature=temperature)
        tried.append({'model': ALT_MODEL, 'attempts': attempts2})
        if ans2 and ans2.strip():
            return {
                'query': query, 
                'answer': ans2.strip(), 
                'docs': idxs.tolist(), 
                'context': context, 
                'route': 'chat-alt', 
                'tried': tried
            }
    
    # All routes failed
    return {
        'query': query, 
        'answer': 'I\'m not sure. (All SDK routes failed)', 
        'docs': idxs.tolist(), 
        'context': context, 
        'route': 'failed', 
        'tried': tried
    }

print('[INFO] SDK generation mode active.')
print(f'       RAW_MODEL = {RAW_MODEL}')
print(f'       ALT_MODEL = {ALT_MODEL}')

[INFO] SDK generation mode active.
       RAW_MODEL = deepseek-r1-distill-qwen-7b-cuda-gpu:0
       ALT_MODEL = deepseek-r1-distill-qwen-7b


In [15]:
# Self-test cell: validates connectivity, embeddings, and answer() basic functionality (SDK mode)
import math, pprint

def rag_self_test(sample_query: str = 'Why use RAG with local inference?', expect_docs: int = 3):
    report = {'base': BASE, 'raw_model': RAW_MODEL, 'alt_model': ALT_MODEL}
    if not BASE:
        report['error'] = 'BASE not resolved'
        return report
    if embedder is None or doc_emb is None:
        report['error'] = 'Embeddings not initialized'
        return report
    if getattr(doc_emb, 'shape', (0,))[0] != len(DOCS):
        report['warning_embeddings'] = f"doc_emb count {getattr(doc_emb,'shape',('?'))} mismatch DOCS {len(DOCS)}"
    try:
        idxs = retrieve(sample_query, k=expect_docs)
        report['retrieved_indices'] = idxs.tolist() if hasattr(idxs, 'tolist') else list(idxs)
    except Exception as e:
        report['error_retrieve'] = str(e)
        return report
    try:
        ans = answer(sample_query, k=expect_docs, max_tokens=80, temperature=0.2)
        report['route'] = ans.get('route')
        report['answer_preview'] = ans.get('answer','')[:160]
        if ans.get('route') == 'failed':
            report['warning_generation'] = 'All SDK routes failed for sample query'
    except Exception as e:
        report['error_generation'] = str(e)
    return report

pprint.pprint(rag_self_test())

{'alt_model': 'deepseek-r1-distill-qwen-7b',
 'answer_preview': 'Okay, so I need to figure out why someone would use '
                   'Retrieval Augmented Generation (RAG) with local inference. '
                   'Let me start by understanding each part of the qu',
 'base': 'http://127.0.0.1:59778',
 'raw_model': 'deepseek-r1-distill-qwen-7b-cuda-gpu:0',
 'retrieved_indices': [0, 3, 1],
 'route': 'chat-first'}


### వివరణ: బ్యాచ్ క్వెరీ స్మోక్ టెస్ట్
`answer()` ద్వారా అనేక ప్రాతినిధ్యాత్మక వినియోగదారు ప్రశ్నలను అమలు చేస్తుంది, తద్వారా నిర్ధారించబడుతుంది:
- రిట్రీవల్ సూచికలు సాధ్యమైన మద్దతు డాక్యుమెంట్లకు సరిపోతాయి.
- ఫాల్బ్యాక్ రూటింగ్ పనిచేస్తుంది (రూట్ విలువ 'failed' కాదు).
- సమాధానాలు గ్రౌండింగ్ సూచనను గౌరవిస్తాయి (హల్యూసినేషన్లు లేవు).
తదుపరి అన్వేషణ కోసం చివరి ఫలిత ఆబ్జెక్టును క్యాప్చర్ చేస్తుంది.


In [16]:
# Quick test queries

queries = [

    "Why use RAG with local inference?",

    "What does vector similarity search do?",

    "Explain privacy benefits."

]



last_result = None

for q in queries:

    try:

        r = answer(q)

        last_result = r

        print(f"Q: {q}\nA: {r['answer']}\nDocs: {r['docs']}\n---")

    except Exception as e:

        print(f"Failed answering '{q}': {e}")



last_result

Q: Why use RAG with local inference?
A: Okay, so I need to figure out why someone would use Retrieval Augmented Generation (RAG) with local inference. Let me start by understanding each part of the question.

First, RAG. From the context given, Doc 1 says that RAG improves answer grounding by injecting relevant context. So RAG is a method that uses retrieval techniques to find the most relevant parts of a document or corpus to augment the generation process. This probably helps in making the generated answers more accurate because they're backed by real data.

Then, local inference. Doc 0 mentions that Foundry Local provides an OpenAI-compatible local inference endpoint. So local inference means running the model on the user's device rather than sending the request to a remote server. This is good for privacy and reducing latency, but it might have limitations in terms of model size or capabilities compared to cloud-based options.

Now, combining RAG with local inference. The context s

{'query': 'Explain privacy benefits.',
 'answer': 'Okay, so I need to explain the privacy benefits mentioned in the provided context. Let me look at the context again. The context includes three documents:\n\nDoc 2 says Edge AI reduces latency and preserves privacy via local execution.\nDoc 3 mentions Small Language Models can offer competitive quality with lower resource usage.\nDoc 1 states Retrieval Augmented Generation improves answer grounding by injecting relevant context.\n\nThe question is about explaining the privacy benefits. So, I should focus on the parts of the context that talk about privacy. \n\nLooking at Doc 2, it mentions Edge AI reduces latency and preserves privacy via local execution. That seems directly related to privacy. I think "local execution" means that the AI processes data on the device itself rather than sending it to a server. This could mean that data doesn\'t have to be transmitted, which might help protect user privacy because it avoids centralizing d

### వివరణ: సింగిల్ ఆంసర్ సౌలభ్య కాల్  
సులభమైన కాపీ/పేస్ట్ పునర్వినియోగం లేదా దిగువ సూచన కోసం తుది త్వరిత సింగిల్-ప్రశ్న కాల్. ముందస్తు వార్మ్-అప్ ప్రశ్నల తర్వాత `answer()` యొక్క ఐడంపొటెంట్ ఉపయోగాన్ని ప్రదర్శిస్తుంది.


In [17]:
result = answer('Why use RAG with local inference?')
result

{'query': 'Why use RAG with local inference?',
 'answer': "Okay, so I need to figure out why someone would use Retrieval Augmented Generation (RAG) with local inference. Let me start by understanding each part of the question.\n\nFirst, RAG. From the context given, Doc 1 says that RAG improves answer grounding by injecting relevant context. So RAG is a method that uses retrieval techniques to find the most relevant parts of a document or corpus to augment the generation process. This probably helps in making the generated answers more accurate because they're backed by real data.\n\nThen, local inference. Doc 0 mentions that Foundry Local provides an OpenAI-compatible local inference endpoint. So local inference means running the model on the user's device rather than sending the request to a remote server. This is good for privacy and reducing latency, but it might have limitations in terms of model size or capabilities compared to cloud-based options.\n\nNow, combining RAG with local

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**అస్పష్టత**:  
ఈ పత్రాన్ని AI అనువాద సేవ [Co-op Translator](https://github.com/Azure/co-op-translator) ఉపయోగించి అనువదించబడింది. మేము ఖచ్చితత్వానికి ప్రయత్నించినప్పటికీ, ఆటోమేటెడ్ అనువాదాల్లో పొరపాట్లు లేదా తప్పిదాలు ఉండవచ్చు. మూల పత్రం దాని స్వదేశీ భాషలో అధికారిక మూలంగా పరిగణించాలి. ముఖ్యమైన సమాచారానికి, ప్రొఫెషనల్ మానవ అనువాదం సిఫార్సు చేయబడుతుంది. ఈ అనువాదం వాడకంలో ఏర్పడిన ఏవైనా అపార్థాలు లేదా తప్పుదారితీసే అర్థాలు కోసం మేము బాధ్యత వహించము.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
